In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
count = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        count += 1
        if count >= 10:
            break
    if count >= 10:
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/.DS_Store
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/.DS_Store
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1947_bacteria_4876.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1952_bacteria_4883.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1951_bacteria_4882.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/.DS_Store


In [2]:
!pip install torch-geometric
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import Dataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, global_mean_pool
from torchvision import transforms
from sklearn.metrics import classification_report, confusion_matrix
from skimage.segmentation import slic
import matplotlib.pyplot as plt
import seaborn as sns

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.9 MB/s eta 0:00:0000:01


In [3]:

labels = ['PNEUMONIA', 'NORMAL']

def process_image(args):
    img, path, label, n_segments = args
    try:
        img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        if img_arr is None:
            return None
        resized_arr = cv2.resize(img_arr, (100, 100))  # Downsample
        
      
        img_color = cv2.cvtColor(resized_arr, cv2.COLOR_GRAY2RGB)#this is done for SLIC MAINLT
        
        # this generates superpixels using SLIC
        segments = slic(img_color, n_segments=n_segments, compactness=40, sigma=1)

        #now we will create node features
        nodes = []
        valid_indices = []
        for i in range(np.max(segments) + 1):
            mask = segments == i
            if np.sum(mask) > 0:
                mean_intensity = np.mean(resized_arr[mask])
                var_intensity = np.var(resized_arr[mask])

                
                grad_x = cv2.Sobel(resized_arr, cv2.CV_64F, 1, 0, ksize=3)#ye edge intensity nikaalega
                grad_y = cv2.Sobel(resized_arr, cv2.CV_64F, 0, 1, ksize=3)
                grad_mag = np.sqrt(grad_x**2 + grad_y**2)
                edge_intensity = np.mean(grad_mag[mask])
                if not np.isnan(mean_intensity) and not np.isnan(var_intensity) and not np.isnan(edge_intensity):
                    nodes.append([mean_intensity / 255.0, var_intensity / 255.0, edge_intensity / 255.0])
                    valid_indices.append(i)
        if not nodes:
            return None
        nodes = torch.tensor(nodes, dtype=torch.float)


        
        edge_index = []
        edge_weight = []
        index_map = {old_idx: new_idx for new_idx, old_idx in enumerate(valid_indices)}
        unique_segments = np.unique(segments)
        for seg_id in unique_segments:
            if seg_id not in valid_indices:
                continue
            mask = segments == seg_id
            dilated = cv2.dilate(mask.astype(np.uint8), np.ones((3, 3), np.uint8), iterations=1)
            neighbors = np.unique(segments[dilated == 1])
            for neighbor_id in neighbors:
                if neighbor_id != seg_id and neighbor_id in valid_indices:
                    edge_index.append([index_map[seg_id], index_map[neighbor_id]])
                    edge_index.append([index_map[neighbor_id], index_map[seg_id]])
                    edge_weight.append(1.0)
                    edge_weight.append(1.0)
        if not edge_index:
            return None
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_weight = torch.tensor(edge_weight, dtype=torch.float)
        
      #this is done for data object creation
        return Data(x=nodes, edge_index=edge_index, edge_attr=edge_weight, y=torch.tensor([labels.index(label)], dtype=torch.long))
    except:
        return None




In [5]:
def get_graph_data(data_dir, n_segments=80, max_images=None):
    #here we can check out and do tuning that which n_segments give best accuracy 
    
    data = []#so this will happen to balance the dataset wwe will do this per class
    for label in labels:
        path = os.path.join(data_dir, label)
        images = os.listdir(path)
        if max_images is not None:
            images = images[:max_images // len(labels)]  
        args = [(img, path, label, n_segments) for img in images]
        with Pool() as pool:
            results = list(tqdm(pool.imap(process_image, args), total=len(args), desc=f"Processing {label} in {data_dir}"))
        data.extend([r for r in results if r is not None])
    return data


In [6]:
from multiprocessing import Pool

train_data = get_graph_data('/kaggle/input/chest-xray-pneumonia/chest_xray/train')
test_data = get_graph_data('/kaggle/input/chest-xray-pneumonia/chest_xray/test')



print(f"Training graphs: {len(train_data)}")
print(f"Test graphs: {len(test_data)}")
print(f"Pneumonia train: {sum(g.y.item() for g in train_data)}")
print(f"Normal train: {len(train_data) - sum(g.y.item() for g in train_data)}")


Processing PNEUMONIA in /kaggle/input/chest-xray-pneumonia/chest_xray/train: 100%|██████████| 3875/3875 [01:25<00:00, 45.43it/s]
Processing NORMAL in /kaggle/input/chest-xray-pneumonia/chest_xray/train: 100%|██████████| 1341/1341 [00:34<00:00, 38.84it/s]
Processing PNEUMONIA in /kaggle/input/chest-xray-pneumonia/chest_xray/test: 100%|██████████| 390/390 [00:08<00:00, 46.84it/s]
Processing NORMAL in /kaggle/input/chest-xray-pneumonia/chest_xray/test: 100%|██████████| 234/234 [00:05<00:00, 42.21it/s]


Training graphs: 5216
Test graphs: 624
Pneumonia train: 1341
Normal train: 3875


In [8]:
class ChestXrayGraphDataset(Dataset):
    def __init__(self, graph_list, augment=False):
        self.graph_list = graph_list
        self.augment = augment

    def __len__(self):
        return len(self.graph_list)

    def __getitem__(self, idx):
        graph = self.graph_list[idx]
        if torch.isnan(graph.x).any():
            print(f"Invalid graph at index {idx}")
            return self.__getitem__((idx + 1) % len(self.graph_list))
        
        if self.augment:
            # Applyint graph augmentation
            if np.random.random() > 0.5:
               
                noise = torch.randn_like(graph.x) * 0.05
                graph.x = graph.x + noise
                graph.x = torch.clamp(graph.x, 0.0, 1.0) 
        return graph
    

In [18]:
class GNNModel(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, dropout_rate=0.3):
        super(GNNModel, self).__init__()
        
        self.conv1 = GATConv(input_dim, hidden_dim, heads=4)
        self.bn1 = nn.BatchNorm1d(hidden_dim * 4)
        
        
        self.conv2 = GATConv(hidden_dim * 4, hidden_dim * 2, heads=2)
        self.bn2 = nn.BatchNorm1d(hidden_dim * 2 * 2)
        
        
        self.conv3 = GATConv(hidden_dim * 2 * 2, hidden_dim, heads=1)
        self.bn3 = nn.BatchNorm1d(hidden_dim)
    
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(dropout_rate)
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
       
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
    
    
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.leaky_relu(x, 0.2)
        x = self.dropout(x)
     
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.leaky_relu(x, 0.2)
        x = self.dropout(x)
        
      
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = F.leaky_relu(x, 0.2)
        
      
        x = global_mean_pool(x, batch)
    
     
        x = self.fc1(x)
        x = F.leaky_relu(x, 0.2)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    

In [30]:
def evaluate(model, loader, criterion):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data).squeeze()
            loss = criterion(out, data.y.float())
            total_loss += loss.item()
            
            pred = (out > 0).float()
            correct += pred.eq(data.y.float()).sum().item()
            total += data.y.size(0)
            
            predictions.extend(pred.cpu().numpy())
            true_labels.extend(data.y.cpu().numpy())
    
    return total_loss / len(loader), correct / total
    
 

In [31]:
def train_with_early_stopping():
   
    train_dataset = ChestXrayGraphDataset(train_data, augment=True)
    test_dataset = ChestXrayGraphDataset(test_data, augment=False)
    
   
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    model = GNNModel().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    
    #class wts are calci for class imbalance
    num_pneumonia = sum(g.y.item() == 0 for g in train_data)
    num_normal = sum(g.y.item() == 1 for g in train_data)
    weight = torch.tensor([num_normal / len(train_data), num_pneumonia / len(train_data)]).to(device)
    
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([num_normal / num_pneumonia]).to(device))
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, min_lr=1e-6, verbose=True)
    
    
    epochs = 20
    best_val_acc = 0
    patience = 4  # Early stopping patience
    patience_counter = 0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data).squeeze()
            loss = criterion(out, data.y.float())
            loss.backward()
            
            # Gradient clipping to prevent exploding gradients(this is one step that i found very useful)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            total_loss += loss.item()
            
    
            pred = (out > 0).float()
            correct += pred.eq(data.y.float()).sum().item()
            total += data.y.size(0)
        
        train_loss = total_loss / len(train_loader)
        train_acc = correct / total
        
        val_loss, val_acc = evaluate(model, test_loader, criterion)#we will
        
        
        scheduler.step(val_acc)#this is done to update learning scheduler
        
       
        history['train_loss'].append(train_loss)#ye history ke liye models ka
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        # Early stopping check
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            # we wil save the best model
            torch.save(model.state_dict(), 'best_gnn_model.pth')
            print(f"Saved new best model with validation accuracy: {val_acc:.4f}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    
    # Load the best model for evaluation
    model.load_state_dict(torch.load('best_gnn_model.pth'))
    

    print("\nFinal evaluation with the best model:")
    test_loss, test_acc = evaluate(model, test_loader, criterion)
    print(f"Final Test Accuracy: {test_acc * 100:.2f}%")
    
    return model, history, test_loader

In [32]:
def get_final_classification_report(model, test_loader):
    model.eval()
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data).squeeze()
            pred = (out > 0).float()
            predictions.extend(pred.cpu().numpy())
            true_labels.extend(data.y.cpu().numpy())
    
    accuracy = np.mean(np.array(predictions) == np.array(true_labels))
    print(f"Final Test Accuracy: {accuracy * 100:.2f}%")
    
    print("\nTHE CLASSIFICATION REPORT OF GNN IS AS FOLLOWS:")
    print(classification_report(true_labels, predictions, target_names=['Pneumonia', 'Normal']))

In [35]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

model,history, test_loader = train_with_early_stopping()
get_final_classification_report(model, test_loader)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20, Train Loss: 0.1827, Train Acc: 0.8052, Val Loss: 0.7105, Val Acc: 0.6987
Saved new best model with validation accuracy: 0.6987
Epoch 2/20, Train Loss: 0.1668, Train Acc: 0.8150, Val Loss: 0.2315, Val Acc: 0.8269
Saved new best model with validation accuracy: 0.8269
Epoch 3/20, Train Loss: 0.1673, Train Acc: 0.8190, Val Loss: 0.2610, Val Acc: 0.8205
Epoch 4/20, Train Loss: 0.1664, Train Acc: 0.8192, Val Loss: 0.4555, Val Acc: 0.7772
Epoch 5/20, Train Loss: 0.1644, Train Acc: 0.8152, Val Loss: 0.3871, Val Acc: 0.7821
Epoch 6/20, Train Loss: 0.1620, Train Acc: 0.8294, Val Loss: 0.3572, Val Acc: 0.8077
Early stopping at epoch 6

Final evaluation with the best model:


/tmp/ipykernel_31/4189575232.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_gnn_model.pth'))


Final Test Accuracy: 82.69%
Final Test Accuracy: 82.69%

THE CLASSIFICATION REPORT OF GNN IS AS FOLLOWS:
              precision    recall  f1-score   support

   Pneumonia       0.83      0.91      0.87       390
      Normal       0.82      0.69      0.75       234

    accuracy                           0.83       624
   macro avg       0.83      0.80      0.81       624
weighted avg       0.83      0.83      0.82       624



# WE HAVE FINALLY DONE IT!!! 
# GOT TEST ACCURACY OF 82.69 Through this GNN Model

# LEARNINGS
I explored a bunch of stuff online how I can get better accuracy through different models
What I did???
Google,Perplexity,stack overflow, documentation-> explored how can i get better accuracy through GNN and not CNN, resnet although I think CNN gives WAYYYYYY BETERRR RESULTS!!
  # here is what I learned through GNN
  1. GTA Over Gnc-> believe me it turned out to be game changer
  2. Tuning of n_Segments
  3. WHAT MISTAKE I WAS MAKING???-> not taking the whole dataset and setting max_images
  4. Weighting the loss function to control the imbalance of the dataset
  5. Gradient clipping was essential to stabilize training and avoid exploding gradients.
  6. Did some hyperparameter tuning with GAT heads
  7. Early stopping and learning rate scheduling ensured optimal convergence without overfitting.
  8. I was stuck at 65-70 with this dataset with GNN Model but after these changes it gives good accuracy